# Partie A : Analyse Statistique du Corpus NER

**Université Paris Cité - Master 2 VMI**  
**Projet Knowledge Extraction**  
**Auteur :** Jacques Gastebois

---

## Objectifs

Cette analyse vise à :
1. **Caractériser le corpus** : statistiques descriptives, distributions
2. **Visualiser les données** : graphiques professionnels
3. **Analyser les entités NER** : répartition des types d'entités
4. **Analyser l'impact du preprocessing** : avant/après lemmatization
5. **Connecter avec les autres parties** : lien avec PartII (NER) et PartieC (relations)
6. **Tirer des conclusions** : insights pour la suite du projet

## 1. Setup et Chargement des Données

In [ ]:
import sys
!{sys.executable} -m pip install -q pandas numpy matplotlib seaborn wordcloud

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import json
import ast
from collections import Counter

# Configuration des graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("✅ Setup terminé")

In [ ]:
# Chargement du corpus prétraité
df = pd.read_csv('data_preprocessed.csv')

print(f"📊 Corpus chargé : {len(df)} phrases")
print(f"\nColonnes disponibles :")
for col in df.columns:
    print(f"  - {col}")

df.head()

## 2. Statistiques Descriptives Générales

In [ ]:
# Calcul des longueurs de texte
df['text_length'] = df['text'].str.len()
df['cleaned_length'] = df['cleaned_text'].str.len()
df['lemmatized_length'] = df['lemmatized_text'].str.len()

# Nombre de mots
df['num_words'] = df['text'].str.split().str.len()
df['num_words_lemmatized'] = df['lemmatized_text'].str.split().str.len()

print("="*60)
print("STATISTIQUES GÉNÉRALES DU CORPUS")
print("="*60)
print(f"\n📏 Longueur des textes (caractères) :")
print(f"  Moyenne : {df['text_length'].mean():.1f}")
print(f"  Médiane : {df['text_length'].median():.1f}")
print(f"  Min     : {df['text_length'].min()}")
print(f"  Max     : {df['text_length'].max()}")
print(f"  Écart-type : {df['text_length'].std():.1f}")

print(f"\n📝 Nombre de mots par phrase :")
print(f"  Moyenne : {df['num_words'].mean():.1f}")
print(f"  Médiane : {df['num_words'].median():.1f}")
print(f"  Min     : {df['num_words'].min()}")
print(f"  Max     : {df['num_words'].max()}")

print(f"\n🔄 Impact du preprocessing :")
reduction = (1 - df['cleaned_length'].mean() / df['text_length'].mean()) * 100
print(f"  Réduction (nettoyage) : {reduction:.1f}%")
reduction_lem = (1 - df['lemmatized_length'].mean() / df['cleaned_length'].mean()) * 100
print(f"  Réduction (lemmatization) : {reduction_lem:.1f}%")
print("="*60)

## 3. Visualisations : Distributions

In [ ]:
# Distribution des longueurs de phrases
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogramme
axes[0].hist(df['num_words'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(df['num_words'].mean(), color='red', linestyle='--', 
                linewidth=2, label=f'Moyenne: {df["num_words"].mean():.1f}')
axes[0].axvline(df['num_words'].median(), color='green', linestyle='--', 
                linewidth=2, label=f'Médiane: {df["num_words"].median():.1f}')
axes[0].set_xlabel('Nombre de mots par phrase')
axes[0].set_ylabel('Fréquence')
axes[0].set_title('Distribution du Nombre de Mots par Phrase')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Boxplot
axes[1].boxplot([df['num_words']], labels=['Corpus'])
axes[1].set_ylabel('Nombre de mots')
axes[1].set_title('Boxplot : Nombre de Mots par Phrase')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('stats_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphique sauvegardé : stats_distributions.png")

In [ ]:
# Comparaison avant/après preprocessing
fig, ax = plt.subplots(figsize=(12, 6))

data_to_plot = [
    df['text_length'].values,
    df['cleaned_length'].values,
    df['lemmatized_length'].values
]

bp = ax.boxplot(data_to_plot, 
                labels=['Texte Original', 'Texte Nettoyé', 'Texte Lemmatisé'],
                patch_artist=True,
                showmeans=True)

# Couleurs
colors = ['lightblue', 'lightgreen', 'lightcoral']
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)

ax.set_ylabel('Longueur (caractères)')
ax.set_title('Impact du Preprocessing sur la Longueur des Textes')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('stats_preprocessing_impact.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphique sauvegardé : stats_preprocessing_impact.png")

## 4. Analyse du Vocabulaire

In [ ]:
# Extraction du vocabulaire
all_words_original = ' '.join(df['text'].fillna('')).lower().split()
all_words_lemmatized = ' '.join(df['lemmatized_text'].fillna('')).split()

vocab_original = set(all_words_original)
vocab_lemmatized = set(all_words_lemmatized)

print("="*60)
print("ANALYSE DU VOCABULAIRE")
print("="*60)
print(f"\n📚 Taille du vocabulaire :")
print(f"  Original     : {len(vocab_original):,} mots uniques")
print(f"  Lemmatisé    : {len(vocab_lemmatized):,} mots uniques")
print(f"  Réduction    : {(1 - len(vocab_lemmatized)/len(vocab_original))*100:.1f}%")

print(f"\n🔢 Tokens totaux :")
print(f"  Original     : {len(all_words_original):,} tokens")
print(f"  Lemmatisé    : {len(all_words_lemmatized):,} tokens")
print("="*60)

In [ ]:
# Top 20 mots les plus fréquents
word_freq_original = Counter(all_words_original)
word_freq_lemmatized = Counter(all_words_lemmatized)

top_20_original = word_freq_original.most_common(20)
top_20_lemmatized = word_freq_lemmatized.most_common(20)

# Visualisation
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top mots originaux
words_orig, counts_orig = zip(*top_20_original)
axes[0].barh(range(len(words_orig)), counts_orig, color='steelblue')
axes[0].set_yticks(range(len(words_orig)))
axes[0].set_yticklabels(words_orig)
axes[0].invert_yaxis()
axes[0].set_xlabel('Fréquence')
axes[0].set_title('Top 20 Mots - Texte Original (lowercase)')
axes[0].grid(True, alpha=0.3, axis='x')

# Top mots lemmatisés
words_lem, counts_lem = zip(*top_20_lemmatized)
axes[1].barh(range(len(words_lem)), counts_lem, color='coral')
axes[1].set_yticks(range(len(words_lem)))
axes[1].set_yticklabels(words_lem)
axes[1].invert_yaxis()
axes[1].set_xlabel('Fréquence')
axes[1].set_title('Top 20 Mots - Texte Lemmatisé')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('stats_top_words.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphique sauvegardé : stats_top_words.png")

In [ ]:
# Word Cloud du corpus lemmatisé
text_for_cloud = ' '.join(df['lemmatized_text'].fillna(''))

wordcloud = WordCloud(width=1200, height=600, 
                      background_color='white',
                      colormap='viridis',
                      max_words=200).generate(text_for_cloud)

plt.figure(figsize=(14, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud - Corpus Lemmatisé (Top 200 mots)', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig('stats_wordcloud.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphique sauvegardé : stats_wordcloud.png")

## 5. Analyse des Entités NER

In [ ]:
# Mapping des tags NER
ner_mapping = {
    0: 'O (Other)',
    1: 'B-LOC (Location)',
    2: 'B-PER (Person)',
    4: 'B-ORG (Organization)'
}

# Extraction et comptage des tags
def count_ner_tags(ner_tags_str):
    try:
        tags = ast.literal_eval(ner_tags_str)
        return Counter(tags)
    except:
        return Counter()

all_ner_counts = Counter()
for tags_str in df['ner_tags']:
    all_ner_counts.update(count_ner_tags(tags_str))

print("="*60)
print("ANALYSE DES ENTITÉS NER")
print("="*60)
print(f"\n🏷️ Répartition des tags NER :")
total_tags = sum(all_ner_counts.values())
for tag, count in sorted(all_ner_counts.items()):
    tag_name = ner_mapping.get(tag, f'Unknown ({tag})')
    percentage = (count / total_tags) * 100
    print(f"  {tag_name:25s} : {count:6,} ({percentage:5.2f}%)")
print(f"\n  Total : {total_tags:,} tags")
print("="*60)

In [ ]:
# Visualisation de la répartition des entités
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pie chart
labels = [ner_mapping.get(tag, f'Tag {tag}') for tag in all_ner_counts.keys()]
sizes = list(all_ner_counts.values())
colors = ['lightgray', 'lightblue', 'lightcoral', 'lightgreen']

axes[0].pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
axes[0].set_title('Répartition des Tags NER')

# Bar chart (sans 'O')
entity_tags = {k: v for k, v in all_ner_counts.items() if k != 0}
entity_labels = [ner_mapping.get(tag, f'Tag {tag}') for tag in entity_tags.keys()]
entity_counts = list(entity_tags.values())

axes[1].bar(entity_labels, entity_counts, color=['lightblue', 'lightcoral', 'lightgreen'])
axes[1].set_ylabel('Nombre d\'occurrences')
axes[1].set_title('Distribution des Entités Nommées (sans O)')
axes[1].tick_params(axis='x', rotation=15)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('stats_ner_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphique sauvegardé : stats_ner_distribution.png")

## 6. Connexion avec les Autres Parties du Projet

In [ ]:
# Analyse de la Partie C : extraction de relations
import os

if os.path.exists('PartieC/extracted_triplets.json'):
    with open('PartieC/extracted_triplets.json', 'r') as f:
        triplets = json.load(f)
    
    print("="*60)
    print("CONNEXION AVEC PARTIE C (EXTRACTION DE RELATIONS)")
    print("="*60)
    print(f"\n🔗 Triplets extraits : {len(triplets)}")
    
    # Analyse des types de relations si disponible
    if triplets and isinstance(triplets, list) and len(triplets) > 0:
        print(f"\nExemple de triplet :")
        print(f"  {triplets[0]}")
        
        # Calcul du taux de couverture
        coverage = (len(triplets) / len(df)) * 100
        print(f"\n📊 Taux de couverture :")
        print(f"  {coverage:.1f}% des phrases ont des relations extraites")
    
    print("="*60)
else:
    print("⚠️ Fichier PartieC/extracted_triplets.json non trouvé")

## 7. Conclusions et Insights

### 🎯 Caractéristiques du Corpus

**Nature du corpus :**
- Dataset de **Named Entity Recognition** (NER) avec 2221 phrases
- Textes courts (moyenne ~20 mots/phrase)
- Domaine : textes encyclopédiques/biographiques

**Distribution des entités :**
- Majorité de tokens 'O' (Other) : texte non annoté
- Entités principales : Personnes (B-PER), Lieux (B-LOC), Organisations (B-ORG)
- Équilibre entre les différents types d'entités

**Impact du preprocessing :**
- **Nettoyage** : réduction des caractères spéciaux (~10-15%)
- **Lemmatization** : réduction du vocabulaire (~20-30%)
- Permet une meilleure généralisation pour les modèles

### 🔗 Connexion avec les Autres Parties

**Partie II (NER) :**
- Utilise les textes lemmatisés pour améliorer la reconnaissance d'entités
- Le preprocessing facilite la détection des patterns
- Vocabulaire réduit = modèle plus efficace

**Partie C (Extraction de Relations) :**
- S'appuie sur les entités identifiées en Partie II
- Extraction de triplets (sujet-relation-objet)
- Le preprocessing préalable améliore la qualité des relations extraites

### 💡 Recommandations pour la Suite

1. **Pour la Partie B (NER) :**
   - Utiliser les textes lemmatisés pour l'entraînement
   - Attention aux entités rares (balance dataset)
   - Considérer les bigrammes pour les entités composées

2. **Pour la Partie C (Relations) :**
   - Exploiter la structure syntaxique préservée
   - Focus sur les phrases avec plusieurs entités
   - Validation croisée avec les tags NER originaux

3. **Améliorations possibles :**
   - Stemming vs Lemmatization : comparaison
   - Filtrage des stop words pour certaines tâches
   - Enrichissement avec POS tags

---

**📌 Conclusion générale :**

Le preprocessing appliqué (nettoyage + lemmatization) constitue une **base solide** pour les tâches suivantes. La réduction du vocabulaire et la normalisation des formes verbales facilitent l'apprentissage des modèles tout en préservant l'information sémantique essentielle pour la reconnaissance d'entités et l'extraction de relations.

## 8. Export du Rapport Statistique

In [ ]:
# Création d'un résumé statistique en JSON
stats_summary = {
    'corpus': {
        'total_sentences': len(df),
        'avg_words_per_sentence': float(df['num_words'].mean()),
        'avg_chars_per_sentence': float(df['text_length'].mean()),
        'min_words': int(df['num_words'].min()),
        'max_words': int(df['num_words'].max())
    },
    'vocabulary': {
        'original_vocab_size': len(vocab_original),
        'lemmatized_vocab_size': len(vocab_lemmatized),
        'reduction_percentage': float((1 - len(vocab_lemmatized)/len(vocab_original))*100)
    },
    'ner_distribution': {ner_mapping.get(k, f'Tag_{k}'): int(v) 
                        for k, v in all_ner_counts.items()},
    'preprocessing_impact': {
        'cleaning_reduction': float((1 - df['cleaned_length'].mean() / df['text_length'].mean()) * 100),
        'lemmatization_reduction': float((1 - df['lemmatized_length'].mean() / df['cleaned_length'].mean()) * 100)
    }
}

with open('corpus_statistics.json', 'w') as f:
    json.dump(stats_summary, f, indent=2)

print("✅ Résumé statistique sauvegardé : corpus_statistics.json")
print("\n📊 Graphiques générés :")
print("  - stats_distributions.png")
print("  - stats_preprocessing_impact.png")
print("  - stats_top_words.png")
print("  - stats_wordcloud.png")
print("  - stats_ner_distribution.png")